In [1]:
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re
from selenium.webdriver.common.keys import Keys
import time
import requests

In [2]:
#크롬 웹드라이버 불러오기
driver = webdriver.Chrome()
res = driver.get('https://m.place.naver.com/place/list?query=제주도관광지&level=top')
driver.implicitly_wait(20)


In [3]:
#2차 크롤링을 위한 bs4 셋팅
session = requests.Session()
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

retries = Retry(total=5,
                backoff_factor=0.1,
                status_forcelist=[ 500, 502, 503, 504 ])

session.mount('http://', HTTPAdapter(max_retries=retries))


In [4]:
#body부분을 잡기 위해 쓸데없이 버튼을 클릭해줌
driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a').click()
driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[1]/a').click()

#검색결과가 모두 보이지 않기 때문에 page down을 눌러 끝까지 펼쳐준다.
for scroll in range(0,30):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)


In [5]:
from bs4 import BeautifulSoup
html = driver.page_source
bs = BeautifulSoup(html, 'html.parser')
soup = bs.select_one('div.YluNG')
naver_info = soup.select('li.VLTHu')



In [37]:
bs.find_all('a', class_='P7gyV')

[<a class="P7gyV" href="https://tivan.naver.com/d/zNr45fTDqO4cbzYj23pBUj9YjaJrr3ZZjTz_GPPiJHWrd6nZccgtUR58MAwe3t-QYB89eyo0Rv-rV7mTdec9txDX4MIN6BuEGny6mqlc5aBmRSt5akiDpPKUYbzHTgbId3lYKJasPPvIPMoFAmbtRyB9fcMdkiwqkiGZqbwX4UfKNZ-AJpIn-ZXCIkySBLM5zFgpjdMvP-OpC1zeUeEUuadziG-P4lt3i1I_55vP4V-3w6kAouzI-yOF_u_OXluwohg4ZG1jNoGx3MrQEkRf5ijuVOqCUP_Z57mOrHIOnqFGQNEN84qmZaowvYG_YTf1RWJR-WpCUCa741GwAMXGt42n6yFglSo3xvByonjS5pnzCCuIJpHPHmfEcGstiNbseH4aSTWAtZ8Vt4D38iPid4HVuEuBiQEMAzjR1Jy27ERONmk_TuNTiyIIaQzIURsLiLogKRP3gA4IlwUmVAs3Ybvj5c-V6Yv4nngBKn7qfc-d1zK8N3woTsvvDWxprB5dOX6zaBiPccX8wN2P_HHaLProTDKrS3rAtO9wypmVukJcfr6IHGGusqzCs0hw-aMibFf3NpYLZuAC-VfpdfbaE79RdEFeGJ37KnIBDFXzRQAOahZj9bxqrmQDL2KuVLQHn9NG0tsYnE9-XzZ5AT0_ZthA6LVuDIw1Ze4oMtQNk7Xwx-ap6DZVxXfhgY6b1NRkPHNk2vRyky0yBJ2JMPlhQw6oZ-2IGxecPKCONVQ-vfiu_KEG-TjXc2GTs350BuUgjxvP4kr5h-renDopARqJcof9aD-AsyF5nlHIZhFSASYUqYbJ9qXxazEvnV1-FM_IVjCrU0f__SMFDZqPSq0MUVHPohhio-mSgMECJ9zC29s=?fu=https%3A%2F%2Fm.place.naver.com%2Fplace%2F1264956495%3Fentry%3Dple%26f

In [6]:
#2차 크롤링을 위한 url
url = 'https://m.place.naver.com'

for info in naver_info[:10]:

    tourist_destination = info.select_one('span.YwYLL').text
    sectors = info.select_one('span.YzBgS').text
    link = info.select_one('div.ouxiq').select_one('a').attrs['href']
    time.sleep(0.06)

    #네이버 플레이스로 이동(place ID로 접속)
    N_res = session.get(link, headers=headers)
    N_soup_srch = BeautifulSoup(N_res.content, 'html.parser')
    mart_oldtel = N_soup_srch.select_one('span.yxkiA > a').attrs['href']
    mart_tel = str(re.sub('tel:', '', mart_oldtel)) # 'tel:' 삭제
    pattern = re.compile("/place/([0-9]+)?c")
    place_ids = []
    place_ids.extend(pattern.findall(driver.page_source))
    #주소는 '공유'에서 파싱
    address = N_soup_srch.select('span.yxkiA > a')[3].attrs['data-line-description']

    print(tourist_destination, '/',sectors,'/',place_ids ,'/', link, '/',  mart_tel, '/', address )
    time.sleep(0.06)


해녀태왁 / 체험여행 / [] / https://tivan.naver.com/d/zNr45fTDqO4cbzYj23pBUj9YjaJrr3ZZjTz_GPPiJHWrd6nZccgtUR58MAwe3t-QYB89eyo0Rv-rV7mTdec9txDX4MIN6BuEGny6mqlc5aBmRSt5akiDpPKUYbzHTgbId3lYKJasPPvIPMoFAmbtRyB9fcMdkiwqkiGZqbwX4UfKNZ-AJpIn-ZXCIkySBLM5zFgpjdMvP-OpC1zeUeEUuadziG-P4lt3i1I_55vP4V-3w6kAouzI-yOF_u_OXluwohg4ZG1jNoGx3MrQEkRf5ijuVOqCUP_Z57mOrHIOnqFGQNEN84qmZaowvYG_YTf1RWJR-WpCUCa741GwAMXGt42n6yFglSo3xvByonjS5pnzCCuIJpHPHmfEcGstiNbseH4aSTWAtZ8Vt4D38iPid4HVuEuBiQEMAzjR1Jy27ERONmk_TuNTiyIIaQzIURsLiLogKRP3gA4IlwUmVAs3Ybvj5c-V6Yv4nngBKn7qfc-d1zK8N3woTsvvDWxprB5dOX6zaBiPccX8wN2P_HHaLProTDKrS3rAtO9wypmVukJcfr6IHGGusqzCs0hw-aMibFf3NpYLZuAC-VfpdfbaE79RdEFeGJ37KnIBDFXzRQAOahZj9bxqrmQDL2KuVLQHn9NG0tsYnE9-XzZ5AT0_ZthA6LVuDIw1Ze4oMtQNk7Xwx-ap6DZVxXfhgY6b1NRkPHNk2vRyky0yBJ2JMPlhQw6oZ-2IGxecPKCONVQ-vfiu_KEG-TjXc2GTs350BuUgjxvP4kr5h-renDopARqJcof9aD-AsyF5nlHIZhFSASYUqYbJ9qXxazEvnV1-FM_IVjCrU0f__SMFDZqPSq0MUVHPohhio-mSgMECJ9zC29s=?fu=https%3A%2F%2Fm.place.naver.com%2Fplace%2F1264956495%3Fentry%3Dple%26from%3

In [7]:
import pandas as pd

# 2차 크롤링을 위한 기본 URL
url = 'https://m.place.naver.com'

# 데이터를 담을 빈 리스트 생성
data = {'Tourist Destination': [],
        'Sectors': [],
        'Link': [],
        'Mart Tel': [],
        'Address': []}

for info in naver_info[:10]:
    # 각 정보 수집
    tourist_destination = info.select_one('span.YwYLL').text
    sectors = info.select_one('span.YzBgS').text
    link = info.select_one('div.ouxiq').select_one('a').attrs['href']
    time.sleep(0.06)

    # 네이버 플레이스로 이동(place ID로 접속)
    N_res = session.get(link, headers=headers)
    N_soup_srch = BeautifulSoup(N_res.content, 'html.parser')
    mart_oldtel = N_soup_srch.select_one('span.yxkiA > a').attrs['href']
    mart_tel = str(re.sub('tel:', '', mart_oldtel))  # 'tel:' 삭제
    
    # 주소는 '공유'에서 파싱
    address = N_soup_srch.select('span.yxkiA > a')[3].attrs['data-line-description']

    # 데이터를 딕셔너리에 추가
    data['Tourist Destination'].append(tourist_destination)
    data['Sectors'].append(sectors)
    data['Link'].append(link)
    data['Mart Tel'].append(mart_tel)
    data['Address'].append(address)

    time.sleep(0.06)

# 데이터프레임으로 변환
df = pd.DataFrame(data)

# 데이터프레임 출력
print(df)


  Tourist Destination  Sectors  \
0                해녀태왁     체험여행   
1            피규어뮤지엄제주      박물관   
2            헬로키티아일랜드     테마파크   
3              동백포레스트  식물원,수목원   
4           아쿠아플라넷 제주    아쿠아리움   
5               카멜리아힐  식물원,수목원   
6                 쇠소깍     강,하천   
7                새별오름       오름   
8               스누피가든    관람,체험   
9                사계해안  해수욕장,해변   

                                                Link        Mart Tel  \
0  https://tivan.naver.com/d/zNr45fTDqO4cbzYj23pB...    064-782-6347   
1  https://tivan.naver.com/d/kvJ1LB8jzKSK4llyrmdp...  0507-1433-2264   
2  https://m.place.naver.com/place/33703960?entry...    064-792-6114   
3  https://m.place.naver.com/place/1611903913?ent...  0507-1331-2102   
4  https://m.place.naver.com/place/19708047?entry...       1833-7001   
5  https://m.place.naver.com/place/11846866?entry...    064-800-6296   
6  https://m.place.naver.com/place/11707633?entry...               #   
7  https://m.place.naver.com/place/17029023?entry

In [8]:
df

,Tourist Destination,Sectors,Link,Mart Tel,Address
0,해녀태왁,체험여행,https://tivan.naver.com/d/zNr45fTDqO4cbzYj23pB...,064-782-6347,제주 제주시 구좌읍 김송로 94
1,피규어뮤지엄제주,박물관,https://tivan.naver.com/d/kvJ1LB8jzKSK4llyrmdp...,0507-1433-2264,제주 서귀포시 안덕면 한창로 243
2,헬로키티아일랜드,테마파크,https://m.place.naver.com/place/33703960?entry...,064-792-6114,제주 서귀포시 안덕면 한창로 340
3,동백포레스트,"식물원,수목원",https://m.place.naver.com/place/1611903913?ent...,0507-1331-2102,제주 서귀포시 남원읍 생기악로 53-38
4,아쿠아플라넷 제주,아쿠아리움,https://m.place.naver.com/place/19708047?entry...,1833-7001,제주 서귀포시 성산읍 섭지코지로 95 아쿠아플라넷 제주
5,카멜리아힐,"식물원,수목원",https://m.place.naver.com/place/11846866?entry...,064-800-6296,제주 서귀포시 안덕면 병악로 166
6,쇠소깍,"강,하천",https://m.place.naver.com/place/11707633?entry...,#,제주 서귀포시 쇠소깍로 104
7,새별오름,오름,https://m.place.naver.com/place/17029023?entry...,#,제주 제주시 애월읍 봉성리 산59-8
8,스누피가든,"관람,체험",https://m.place.naver.com/place/1875149782?ent...,064-903-1111,제주 제주시 구좌읍 금백조로 930
9,사계해안,"해수욕장,해변",https://m.place.naver.com/place/17083214?entry...,#,제주 서귀포시 안덕면 사계리


In [44]:
pattern = re.compile("place/([0-9]+)")
player_ids = []

In [50]:
test = bs.find_all('a', class_='P7gyV')

In [52]:
from tqdm import tqdm

In [83]:
test_res = []
test_str = []
for x in tqdm(test[:11]):
    str_x = str(x).split('/')[4].split('?')[0]
    print(str_x)
    if(str_x[0].isdigit() != True):
        continue
    if str_x not in test_res:
        test_res.append(str_x)
print('-'*50)
print(test_res)

100%|██████████| 11/11 [00:00<00:00, 3820.58it/s]

zNr45fTDqO4cbzYj23pBUj9YjaJrr3ZZjTz_GPPiJHWrd6nZccgtUR58MAwe3t-QYB89eyo0Rv-rV7mTdec9txDX4MIN6BuEGny6mqlc5aBmRSt5akiDpPKUYbzHTgbId3lYKJasPPvIPMoFAmbtRyB9fcMdkiwqkiGZqbwX4UfKNZ-AJpIn-ZXCIkySBLM5zFgpjdMvP-OpC1zeUeEUuadziG-P4lt3i1I_55vP4V-3w6kAouzI-yOF_u_OXluwohg4ZG1jNoGx3MrQEkRf5ijuVOqCUP_Z57mOrHIOnqFGQNEN84qmZaowvYG_YTf1RWJR-WpCUCa741GwAMXGt42n6yFglSo3xvByonjS5pnzCCuIJpHPHmfEcGstiNbseH4aSTWAtZ8Vt4D38iPid4HVuEuBiQEMAzjR1Jy27ERONmk_TuNTiyIIaQzIURsLiLogKRP3gA4IlwUmVAs3Ybvj5c-V6Yv4nngBKn7qfc-d1zK8N3woTsvvDWxprB5dOX6zaBiPccX8wN2P_HHaLProTDKrS3rAtO9wypmVukJcfr6IHGGusqzCs0hw-aMibFf3NpYLZuAC-VfpdfbaE79RdEFeGJ37KnIBDFXzRQAOahZj9bxqrmQDL2KuVLQHn9NG0tsYnE9-XzZ5AT0_ZthA6LVuDIw1Ze4oMtQNk7Xwx-ap6DZVxXfhgY6b1NRkPHNk2vRyky0yBJ2JMPlhQw6oZ-2IGxecPKCONVQ-vfiu_KEG-TjXc2GTs350BuUgjxvP4kr5h-renDopARqJcof9aD-AsyF5nlHIZhFSASYUqYbJ9qXxazEvnV1-FM_IVjCrU0f__SMFDZqPSq0MUVHPohhio-mSgMECJ9zC29s=
zNr45fTDqO4cbzYj23pBUj9YjaJrr3ZZjTz_GPPiJHWrd6nZccgtUR58MAwe3t-QYB89eyo0Rv-rV7mTdec9txDX4MIN6BuEGny6mqlc5aBmRSt5akiDpPKUYbz

In [78]:
test_res.__len__()

4

In [48]:
bs.find_all('a', class_='P7gyV')

[<a class="P7gyV" href="https://tivan.naver.com/d/zNr45fTDqO4cbzYj23pBUj9YjaJrr3ZZjTz_GPPiJHWrd6nZccgtUR58MAwe3t-QYB89eyo0Rv-rV7mTdec9txDX4MIN6BuEGny6mqlc5aBmRSt5akiDpPKUYbzHTgbId3lYKJasPPvIPMoFAmbtRyB9fcMdkiwqkiGZqbwX4UfKNZ-AJpIn-ZXCIkySBLM5zFgpjdMvP-OpC1zeUeEUuadziG-P4lt3i1I_55vP4V-3w6kAouzI-yOF_u_OXluwohg4ZG1jNoGx3MrQEkRf5ijuVOqCUP_Z57mOrHIOnqFGQNEN84qmZaowvYG_YTf1RWJR-WpCUCa741GwAMXGt42n6yFglSo3xvByonjS5pnzCCuIJpHPHmfEcGstiNbseH4aSTWAtZ8Vt4D38iPid4HVuEuBiQEMAzjR1Jy27ERONmk_TuNTiyIIaQzIURsLiLogKRP3gA4IlwUmVAs3Ybvj5c-V6Yv4nngBKn7qfc-d1zK8N3woTsvvDWxprB5dOX6zaBiPccX8wN2P_HHaLProTDKrS3rAtO9wypmVukJcfr6IHGGusqzCs0hw-aMibFf3NpYLZuAC-VfpdfbaE79RdEFeGJ37KnIBDFXzRQAOahZj9bxqrmQDL2KuVLQHn9NG0tsYnE9-XzZ5AT0_ZthA6LVuDIw1Ze4oMtQNk7Xwx-ap6DZVxXfhgY6b1NRkPHNk2vRyky0yBJ2JMPlhQw6oZ-2IGxecPKCONVQ-vfiu_KEG-TjXc2GTs350BuUgjxvP4kr5h-renDopARqJcof9aD-AsyF5nlHIZhFSASYUqYbJ9qXxazEvnV1-FM_IVjCrU0f__SMFDZqPSq0MUVHPohhio-mSgMECJ9zC29s=?fu=https%3A%2F%2Fm.place.naver.com%2Fplace%2F1264956495%3Fentry%3Dple%26f

In [45]:
place_ids.extend(pattern.findall(bs.find_all('a', class_='P7gyV')[5]))

TypeError: expected string or bytes-like object, got 'Tag'

In [10]:
place_ids.extend(pattern.findall(driver.page_source))
place_ids

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '0068',
 '',
 '',
 '',
 '',
 '0068',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '

In [ ]:
import requests

from selenium import webdriver

from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By

import time

import json

: 